In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install pyrsgis

In [0]:
######################### Import des libraries #######################

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader, sampler
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from copy import deepcopy
import torch
import torch.nn as nn
import pandas as pd
import scipy.signal
import scipy.ndimage
import os
import numpy as np

Warning! matplotlib_scalebar library not found.


In [0]:
######################### Import des données (1 seule tile) #########################

path = "/content/drive/My Drive/D4G/"

Test = "2_Image_Seredou_32bits_20170205/S2A_20170205_seredou_32bits.tif"
MSI = "2_MSI_Seredou_20170205/S2A_20170205_seredou_ZE_MSI_89.tif"
Truth = "GroundTruth_Seredou_20170205/GroundTruth_20170205_seredou_ZE_89.tif"
"""
path = "/content/drive/My Drive/D4G/4_Congo_20200128/"

Test = "Image_Congo_20200128.tif"
MSI = "MSI_Congo_20200128.tif"
Truth = "GroundTruth_Congo_20200128.tif"""

'\npath = "/content/drive/My Drive/D4G/4_Congo_20200128/"\n\nTest = "Image_Congo_20200128.tif"\nMSI = "MSI_Congo_20200128.tif"\nTruth = "GroundTruth_Congo_20200128.tif'

In [0]:
######################### Import des données (tout en 1 seule tile) #########################

path = "/content/drive/My Drive/D4G/Seredou_20190307_v2.tif"

In [0]:
######################### Import des données (plusieurs tiles) #########################

os.chdir("/content/drive/My Drive/D4G/1_Seredou_2015/")

#for testing
"""MSI = ['1_MSI_Seredou_2015_tiled\\S2A_20151203_MSI_1km2_1.tif']
test = ['1_Images_Seredou_2015_16bit_tiled\\S2A_20151203_seredou_1km2_1.tif']
truth = ['1_Mask_Seredou_2015_tiled\\GroundTruth_seredou_20151203_1km2_1.tif']"""

#for prod
test_raw = os.listdir("1_Images_Seredou_2015_32bits")
MSI_raw = os.listdir("1_MSI_Seredou_2015")
truth_raw = os.listdir("1_GroundTruth_Seredou_2015_tiled")

Test = {int(k.split("_")[-1].split(".tif")[0]):('1_Images_Seredou_2015_32bits/'+k) for k in test_raw}
MSI = {int(k.split("_")[-1].split(".tif")[0]):('1_MSI_Seredou_2015/'+k) for k in MSI_raw}
Truth = {int(k.split("_")[-1].split(".tif")[0].split("(")[0]):('1_GroundTruth_Seredou_2015_tiled/'+k) for k in truth_raw}

In [0]:
######################### Fonctions auxiliaires #########################

def divide_test_in_squares(size, img):
    return [img[:,int(i % img.shape[0])*size:((int(i % img.shape[0])+1)*size), int(i // img.shape[0])*size:(int(i // img.shape[0])+1)*size] for i in range(int(img.shape[0] * img.shape[1] // size**2))]

def divide_image_in_squares(size, img):
    return [img[int(i % img.shape[0])*size:((int(i % img.shape[0])+1)*size), int(i // img.shape[0])*size:(int(i // img.shape[0])+1)*size] for i in range(int(img.shape[0] * img.shape[1] // size**2))]

def get_window(img, center, size):
    
    window = torch.tensor([0. for i in range(size*size)])
    if center[0] + size // 2 < img.shape[0]:
        i_begin = max(0,int(center[0]-size//2))
        i_end = i_begin + size
    else:
        i_end = img.shape[0]
        i_begin = i_end - size
    if center[1] + size // 2 < img.shape[1]:
        j_begin = max(0,int(center[1]-size//2))
        j_end = j_begin + size
    else:
        j_end = img.shape[1]
        j_begin = j_end - size

    for i in range(i_begin, i_end):
        for j in range(j_begin, j_end):
            window[i - i_begin + size*(j - j_begin)] = float(img[i,j])
    return window

In [0]:
######################### Base de données (1 image) #########################

class OurDataset(Dataset):
    """This dataset includes .... """
    
    def __init__(self, path, Test, MSI, Truth, size, percentage, transform=None):
        """
        Args:
            path: (str) path to the images directory.
        """
        #get data
        ds1, data_MSI = raster.read(path + MSI, bands='all')
        ds2, data_test = raster.read(path + Test, bands='all')
        ds3, data_truth = raster.read(path + Truth, bands='all')

        data_MSI[data_MSI < 0] = 0
        data_test[data_test < 0] = 0
        data_truth[data_truth < 0] = 0

        self.MSI = divide_image_in_squares(size, data_MSI)
        self.Test = divide_test_in_squares(size, data_test)
        self.Truth = divide_image_in_squares(size, data_truth)
  
        self.size = size
        self.mode = "train"
        self.id_train, self.id_test = train_test_split([i for i in range(len(self.Test))], test_size = percentage, random_state=42, shuffle=True)
        
    def __len__(self):
        if self.mode == "train":
            return len(self.id_train)
        else:
            return len(self.id_test)

    def __getitem__(self, id):
        """
        Args:
            idx: (int) the index of the subject/session whom data is loaded.
        Returns:
            sample: (dict) corresponding data described by the following keys:
                scan: 11 channels value
                mask: true value
        """
        if self.mode == "train":
            idx = self.id_train[id]
        else:
            idx = self.id_test[id]

        #get data
        MSI = self.MSI[idx]
        Test = self.Test[idx]
        Truth = self.Truth[idx]

        feature_data = torch.tensor([[0. for i in range(11*10)] for j in range(Test.shape[1] * Test.shape[2])])
        feature_truth = np.array([0 for j in range(Test.shape[1] * Test.shape[2])])

        #On importe les données
        for i in range(Test.shape[1]):
            for j in range(Test.shape[2]):
                feature_truth[i+Test.shape[1]*j] = Truth[i,j]
                
                msi = get_window(MSI, (i,j), self.size // 2)
                if msi.max() != 0:
                    msi = msi/msi.max()
                laplace_gauss = torch.tensor(scipy.ndimage.gaussian_laplace(msi.numpy(), sigma = 1))

                feature_data[i,j,100] = float(MSI[i,j])
                feature_data[i,j,101] = msi.max()
                feature_data[i,j,102] = msi.min()
                feature_data[i,j,103] = msi.mean()
                feature_data[i,j,104] = msi.std()
                feature_data[i,j,105] = msi.median()
                feature_data[i,j,106] = laplace_gauss.min()
                feature_data[i,j,107] = laplace_gauss.max()
                feature_data[i,j,108] = laplace_gauss.mean()
                feature_data[i,j,109] = laplace_gauss.std()
                
                for k in range (11):
                    test = get_window(Test[k,:,:], (i,j), 5)
                    if test.max() != 0:
                        test = test/test.max()

                    laplace_gauss = torch.tensor(scipy.ndimage.gaussian_laplace(get_window(Test[k,:,:], (i,j), 11).numpy(), sigma = 1))

                    feature_data[i+Test.shape[1]*j,k*10] = float(Test[k,i,j])
                    feature_data[i+Test.shape[1]*j,k*10 + 1] = test.max()
                    feature_data[i+Test.shape[1]*j,k*10 + 2] = test.min()
                    feature_data[i+Test.shape[1]*j,k*10 + 3] = test.mean()
                    feature_data[i+Test.shape[1]*j,k*10 + 4] = test.std()
                    feature_data[i+Test.shape[1]*j,k*10 + 5] = test.median()
                    feature_data[i+Test.shape[1]*j,k*10 + 6] = laplace_gauss.max()
                    feature_data[i+Test.shape[1]*j,k*10 + 7] = laplace_gauss.min()
                    feature_data[i+Test.shape[1]*j,k*10 + 8] = laplace_gauss.mean()
                    feature_data[i+Test.shape[1]*j,k*10 + 9] = laplace_gauss.std()
         
        sample = {'data': feature_data, 'mask': feature_truth}

        return sample

    def train(self):
        """Put all the transforms of the dataset in training mode"""
        self.transform.train()

    def set_mode(self, mode):
        """Change mode of the database"""
        self.mode = mode

    def eval(self):
        """Put all the transforms of the dataset in evaluation mode"""
        self.transform.eval()

In [0]:
######################### Base de données (tout en 1 image) #########################

class OurDataset(Dataset):
    """This dataset includes .... """
    
    def __init__(self, path, size, percentage, transform=None):
        """
        Args:
            path: (str) path to the images directory.
        """
        #get data
        ds1, data = raster.read(path, bands='all')

        data[data < 0] = 0

        self.Test = divide_test_in_squares(size, data[:11,:,:])
        self.Truth = divide_image_in_squares(size, data[11,:,:])
        self.size = size
        self.mode = "train"
        self.id_train, self.id_test = train_test_split([i for i in range(len(self.Test))], test_size = percentage, random_state=42, shuffle=True)
        
    def __len__(self):
        if self.mode == "train":
            return len(self.id_train)
        else:
            return len(self.id_test)

    def __getitem__(self, id):
        """
        Args:
            idx: (int) the index of the subject/session whom data is loaded.
        Returns:
            sample: (dict) corresponding data described by the following keys:
                scan: 11 channels value
                mask: true value
        """
        if self.mode == "train":
            idx = self.id_train[id]
        else:
            idx = self.id_test[id]

        Test = self.Test[idx]
        Truth = self.Truth[idx]

        feature_data = torch.tensor([[0. for i in range(11*10)] for j in range(Test.shape[1] * Test.shape[2])])
        feature_truth = np.array([0 for j in range(Test.shape[1] * Test.shape[2])])

        #On importe les données
        for i in range(Test.shape[1]):
            for j in range(Test.shape[2]):
                feature_truth[i+Test.shape[1]*j] = Truth[i,j]
                
                for k in range (11):
                    test = get_window(Test[k,:,:], (i,j), 5)
                    if test.max() != 0:
                        test = test/test.max()

                    laplace_gauss = torch.tensor(scipy.ndimage.gaussian_laplace(get_window(Test[k,:,:], (i,j), 11).numpy(), sigma = 1))

                    feature_data[i+Test.shape[1]*j,k*10] = float(Test[k,i,j])
                    feature_data[i+Test.shape[1]*j,k*10 + 1] = test.max()
                    feature_data[i+Test.shape[1]*j,k*10 + 2] = test.min()
                    feature_data[i+Test.shape[1]*j,k*10 + 3] = test.mean()
                    feature_data[i+Test.shape[1]*j,k*10 + 4] = test.std()
                    feature_data[i+Test.shape[1]*j,k*10 + 5] = test.median()
                    feature_data[i+Test.shape[1]*j,k*10 + 6] = laplace_gauss.max()
                    feature_data[i+Test.shape[1]*j,k*10 + 7] = laplace_gauss.min()
                    feature_data[i+Test.shape[1]*j,k*10 + 8] = laplace_gauss.mean()
                    feature_data[i+Test.shape[1]*j,k*10 + 9] = laplace_gauss.std()
         
        sample = {'data': feature_data, 'mask': feature_truth}

        return sample

    def train(self):
        """Put all the transforms of the dataset in training mode"""
        self.transform.train()

    def set_mode(self, mode):
        """Change mode of the database"""
        self.mode = mode

    def eval(self):
        """Put all the transforms of the dataset in evaluation mode"""
        self.transform.eval()

In [0]:
######################### Base de données (plusieurs images) #########################

class OurDataset(Dataset):
    """This dataset includes .... """
    
    def __init__(self, path, Test, MSI, Truth, percentage, transform=None):
        """
        Args:
            path: (str) path to the images directory.
        """
        self.MSI = {}
        self.Test = {}
        self.Truth = {}
        for key in MSI.keys():
            _, self.MSI[key] = raster.read(MSI[key], bands='all')
            _, self.Test[key] = raster.read(Test[key], bands='all')
            _, self.Truth[key] = raster.read(Truth[key], bands='all')

        self.mode = "train"
        self.id_train, self.id_test = train_test_split([i for i in range(len(list(self.Test.keys())))], test_size = percentage, random_state=42, shuffle=True)
        
    def __len__(self):
        if self.mode == "train":
            return len(self.id_train)
        else:
            return len(self.id_test)

    def __getitem__(self, id):
        """
        Args:
            idx: (int) the index of the subject/session whom data is loaded.
        Returns:
            sample: (dict) corresponding data described by the following keys:
                scan: 11 channels value
                mask: true value
        """

        if self.mode == "train":
            idx = self.id_train[id] + 1
        else:
            idx = self.id_test[id] + 1

        #get data
        MSI = self.MSI[idx]
        Test = self.Test[idx]
        Truth = self.Truth[idx]
        
        MSI[MSI < 0] = 0
        Test[Test < 0] = 0
        Truth[Truth < 0] = 0
        
        feature_data = torch.tensor([[0. for i in range(11*10)] for j in range(Test.shape[1] * Test.shape[2])])
        feature_truth = np.array([0 for j in range(Test.shape[1] * Test.shape[2])])

        #On importe les données
        for i in range(Test.shape[1]):
            for j in range(Test.shape[2]):
                feature_truth[i+Test.shape[1]*j] = Truth[i,j]
                
                msi = get_window(MSI, (i,j), self.size // 2)
                if msi.max() != 0:
                    msi = msi/msi.max()
                laplace_gauss = torch.tensor(scipy.ndimage.gaussian_laplace(msi.numpy(), sigma = 1))

                feature_data[i,j,100] = float(MSI[i,j])
                feature_data[i,j,101] = msi.max()
                feature_data[i,j,102] = msi.min()
                feature_data[i,j,103] = msi.mean()
                feature_data[i,j,104] = msi.std()
                feature_data[i,j,105] = msi.median()
                feature_data[i,j,106] = laplace_gauss.min()
                feature_data[i,j,107] = laplace_gauss.max()
                feature_data[i,j,108] = laplace_gauss.mean()
                feature_data[i,j,109] = laplace_gauss.std()
                
                for k in range (11):
                    test = get_window(Test[k,:,:], (i,j), 5)
                    if test.max() != 0:
                        test = test/test.max()

                    laplace_gauss = torch.tensor(scipy.ndimage.gaussian_laplace(get_window(Test[k,:,:], (i,j), 11).numpy(), sigma = 1))

                    feature_data[i+Test.shape[1]*j,k*10] = float(Test[k,i,j])
                    feature_data[i+Test.shape[1]*j,k*10 + 1] = test.max()
                    feature_data[i+Test.shape[1]*j,k*10 + 2] = test.min()
                    feature_data[i+Test.shape[1]*j,k*10 + 3] = test.mean()
                    feature_data[i+Test.shape[1]*j,k*10 + 4] = test.std()
                    feature_data[i+Test.shape[1]*j,k*10 + 5] = test.median()
                    feature_data[i+Test.shape[1]*j,k*10 + 6] = laplace_gauss.max()
                    feature_data[i+Test.shape[1]*j,k*10 + 7] = laplace_gauss.min()
                    feature_data[i+Test.shape[1]*j,k*10 + 8] = laplace_gauss.mean()
                    feature_data[i+Test.shape[1]*j,k*10 + 9] = laplace_gauss.std()
         
        sample = {'data': feature_data, 'mask': feature_truth}

        return sample

    def train(self):
        """Put all the transforms of the dataset in training mode"""
        self.transform.train()

    def set_mode(self, mode):
        """Change mode of the database"""
        self.mode = mode

    def eval(self):
        """Put all the transforms of the dataset in evaluation mode"""
        self.transform.eval()

In [0]:
def train(model, train_loader, criterion, optimizer, n_epochs, size):
    """
    Method used to train the LNN
    
    Args:
        model: (nn.Module) the neural network
        train_loader: (DataLoader) a DataLoader wrapping the dataset
        criterion: (nn.Module) a method to compute the loss of a mini-batch of images
        optimizer: (torch.optim) an optimization algorithm
        n_epochs: (int) number of epochs performed during training

    Returns:
        best_model: (nn.Module) the trained neural network
    """
    best_model = deepcopy(model)
    train_best_loss = np.inf

    batch_size = train_loader.batch_size
    n = 10

    n_batches = n//batch_size

    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        total_eq = 0
        total_nb = 0

        for i, data in enumerate(train_loader):
            images, mask = torch.reshape(data['data'], ((size**2)*batch_size,110)), data['mask']
            scans = torch.flatten(mask) - 1
            outputs = torch.reshape(model(images), (scans.shape[0],3))
            loss = criterion(outputs, scans)
            loss.backward()
            soft_outputs = nn.functional.softmax(outputs)
            total_eq += int(torch.sum(torch.argmax(soft_outputs,1) == scans))
            total_loss += loss.item()
            total_nb += scans.shape[0]
            optimizer.step()
            optimizer.zero_grad()
            
        mean_loss = total_loss / len(train_loader.dataset)
        print('Epoch %i: loss = %f & accuracy = %f' % (epoch, mean_loss, total_eq/total_nb))

        if mean_loss < train_best_loss:
            best_model = deepcopy(model)
            train_best_loss = mean_loss
    
    return best_model

def test(model, data_loader, criterion, size):
    """
    Method used to test a LNN
    
    Args:
        model: (nn.Module) the neural network
        data_loader: (DataLoader) a DataLoader wrapping the dataset
        criterion: (nn.Module) a method to compute the loss of a mini-batch of images
    """
    model.eval()

    batch_size = data_loader.batch_size
    n = 10000

    n_batches = n//batch_size
    nb_true = 0
    nb_total = 0
    size_loss = 0
    total_loss = 0
    
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            images, mask = torch.reshape(data['data'], ((size**2)*batch_size,110)), data['mask']
            scans = torch.flatten(mask) - 1
            outputs = torch.reshape(model(images), (scans.shape[0],3))
            loss = criterion(outputs, scans)
            soft_outputs = nn.functional.softmax(outputs)
            total_loss += loss.item()
            size_loss += batch_size
            nb_true += int(torch.sum(torch.argmax(outputs,1) == scans))
            nb_total += scans.shape[0]

    print("Final loss : {} & accuracy : {}".format(str(total_loss/ size_loss), str(nb_true/nb_total)))

In [0]:
########################## Modèle ############################

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.lin1 = nn.Linear(110, 110)
        self.lin2 = nn.Linear(110, 110)
        self.lin3 = nn.Linear(110, 110)
        self.lin4 = nn.Linear(110, 110)
        self.lin5 = nn.Linear(110, 3)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        x = self.relu(x)
        x = self.lin3(x)
        x = self.relu(x)
        x = self.lin4(x)
        x = self.relu(x)
        x = self.lin5(x)
        return nn.functional.softmax(x)

In [0]:
########################## Database ############################

size = 20
#database = OurDataset(path, Test, MSI, Truth, 0.2)
database = OurDataset(path, size, 0.2)

In [0]:
########################## Prétraitement ############################

batch_size = 5

database.set_mode("train")
dataloader_train = DataLoader(database, batch_size=batch_size, drop_last=True)

model = Model()
criterion = nn.MultiMarginLoss(p=1)
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.07)
n_epochs = 5

In [0]:
#################### Entrainement du modèle ##########################

train(model, dataloader_train, criterion, optimizer, n_epochs, size)
torch.save(model.state_dict(), "model")

In [0]:
############################### Statistiques ################################

#Predict for test data 

database.set_mode("test")
dataloader_val = DataLoader(database, batch_size=batch_size, drop_last=True)

test(model, dataloader_val, criterion, size)

Final loss : 0.05909258921941121 & accuracy : 0.4681666666666667
